<a href="https://colab.research.google.com/github/karabhilash/CS6910-Fun_of_DL/blob/main/Assignment2/PartB/Assignment2_PartB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras

# keras pre-trained models
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2 
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras import layers
from keras.layers import Dense, Flatten, Conv2D, Activation , GlobalAveragePooling2D,Dropout
from keras.models import Sequential,  Model

!pip install wandb

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
!mkdir -p '/gdrive/My Drive/CS6910-A2/'
%cd '/gdrive/My Drive/CS6910-A2/'

Mounted at /gdrive
/gdrive/My Drive/CS6910-A2


In [ ]:



!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
!unzip nature_12K.zip
!rm nature_12K.zip


In [ ]:
def build_model(neurons, num_classes,optimizer,base_model,image_height,image_width):
    #ignore the last dense layer in base model. All base model have a average pooling layer prior to last layer with the name avg_pool
    base = base_model(weights="imagenet",include_top=False,pooling='avg')
    # freeze base layers
    for layer in base.layers:
      layer.trainable = False
    
      
    input = keras.Input(shape=(image_height,image_width,3))
    x = base(input,training=False)#for avoiding fine-tuning the batch normalization layer,we run on inference mode

    # attach a fine-tuning layer
    x = Dense(neurons, activation='relu')(x)
    x = Dropout(0.3)(x)
    guesses = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input, outputs=guesses)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model,base


def fine_tune(model,base,magik_number):
    for layer in base.layers[:magik_number]:
      layer.trainable = False
    for layer in base.layers[magik_number:]:
      layer.trainable = True


    model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model



In [ ]:

import wandb
from wandb.keras import WandbCallback

sweep_config = {
  "method": "bayes",
  "metric":{
  "name": "val_accuracy",
  "goal": "maximize"
  },
  'early_terminate': {
        'type':'hyperband',
        'min_iter': 4, #since max_epochs =5
        's': 1
  },
  "parameters": {
        

        "data_augmentation": {
            "values": [True, False]
        },
        "optimizer": {
            "values": [ "adam", "rmsprop"]
        },

        "base_model": {
            "values": ['InceptionV3', 'InceptionResNetV2','ResNet50','Xception','NASNetLarge']
        },
        "neurons": {
            "values": [128,256,512]
        }

    }
}

sweep_id = wandb.sweep(sweep_config,project='CS6910-Assignment2-1')



<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 89794rxn
Sweep URL: https://wandb.ai/abhilash-kar0/CS6910-Assignment2-1/sweeps/89794rxn


In [ ]:
model_dict = dict(zip(['InceptionV3', 'InceptionResNetV2','ResNet50','Xception'],[InceptionV3, InceptionResNetV2,ResNet50,Xception]))
magic_num_dict = dict(zip(['InceptionV3', 'InceptionResNetV2','ResNet50','Xception'],[279,758,179,115]))

In [ ]:
def ParamTuning():

    conf_defaults=dict(
        base_model = 'InceptionV3',
        data_augmentation=True,
        optimizer='adam',
        neurons=256,
        epochs=5,
        batch_size=32
    )
    wandb.init(config=conf_defaults)
    CONFIG = wandb.config


    Img_height, Img_width= 224,224
    num_classes=10



    
    
    train_generator = None
    if CONFIG.data_augmentation:
        train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            zca_epsilon=1e-06,
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1,
            brightness_range=None,
            shear_range=0.1,
            zoom_range=0.1,
            channel_shift_range=0.0,
            fill_mode="nearest",
            cval=0.0,
            horizontal_flip=True,
            vertical_flip=False,
            rescale=1/255.0,
            preprocessing_function=None,
            data_format=None,
            validation_split=0.1,
            dtype=None,
        )
    else :
        train_generator=  tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.1,rescale=1/255.0)
    test_generator=  tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)


    train_gen = train_generator.flow_from_directory(
        './inaturalist_12K/train',
        subset='training',
        target_size=(Img_height, Img_width),
        batch_size=CONFIG.batch_size,
        class_mode='categorical')
    val_gen = train_generator.flow_from_directory(
        './inaturalist_12K/train',
        subset='validation',
        target_size=(Img_height, Img_width),
        batch_size=CONFIG.batch_size,
        class_mode='categorical')
    test_gen = test_generator.flow_from_directory(
        './inaturalist_12K/val',
        target_size=(Img_height, Img_width),
        batch_size=CONFIG.batch_size,
        class_mode='categorical')
    model,base = build_model(CONFIG.neurons,num_classes,CONFIG.optimizer,model_dict[CONFIG.base_model],Img_height, Img_width)
    
    h = model.fit(x=train_gen,validation_data = val_gen,
                        steps_per_epoch = train_gen.samples // CONFIG.batch_size,
                        validation_steps = val_gen.samples // CONFIG.batch_size,
                        epochs = CONFIG.epochs, 
                        callbacks=[WandbCallback()])
    #model.save('model'+str(CONFIG))
    wandb.finish()
    return h

In [ ]:
def Finetuning():

    conf_defaults=dict(
        base_model = 'InceptionV3',
        data_augmentation=True,
        optimizer='adam',
        neurons=256,
        epochs=5,
        batch_size=32
    )
    wandb.init(config=conf_defaults)
    CONFIG = wandb.config


    Img_height, Img_width= 224,224
    num_classes=10


    
    
    train_generator = None
    if CONFIG.data_augmentation:
        train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            zca_epsilon=1e-06,
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1,
            brightness_range=None,
            shear_range=0.1,
            zoom_range=0.1,
            channel_shift_range=0.0,
            fill_mode="nearest",
            cval=0.0,
            horizontal_flip=True,
            vertical_flip=False,
            rescale=1/255.0,
            preprocessing_function=None,
            data_format=None,
            validation_split=0.1,
            dtype=None,
        )
    else :
        train_generator=  tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.1,rescale=1/255.0)
    test_generator=  tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)


    train_gen = train_generator.flow_from_directory(
        './inaturalist_12K/train',
        subset='training',
        target_size=(Img_height, Img_width),
        batch_size=CONFIG.batch_size,
        class_mode='categorical')
    val_gen = train_generator.flow_from_directory(
        './inaturalist_12K/train',
        subset='validation',
        target_size=(Img_height, Img_width),
        batch_size=CONFIG.batch_size,
        class_mode='categorical')
    test_gen = test_generator.flow_from_directory(
        './inaturalist_12K/val',
        target_size=(Img_height, Img_width),
        batch_size=CONFIG.batch_size,
        class_mode='categorical')
    model,base = build_model(CONFIG.neurons,num_classes,CONFIG.optimizer,model_dict[CONFIG.base_model],Img_height, Img_width)
    h = model.fit(x=train_gen,validation_data = val_gen,
                        steps_per_epoch = train_gen.samples // CONFIG.batch_size,
                        validation_steps = val_gen.samples // CONFIG.batch_size,
                        epochs = CONFIG.epochs
                        )
    model = fine_tune(model,base,magic_num_dict[CONFIG.base_model])
    h2 = model.fit(x=train_gen,validation_data = val_gen,
                        steps_per_epoch = train_gen.samples // CONFIG.batch_size,
                        validation_steps = val_gen.samples // CONFIG.batch_size,
                        epochs = CONFIG.epochs, 
                        callbacks=[WandbCallback()])
    #model.save('model'+str(CONFIG))
    wandb.finish()
    return h

In [ ]:
Finetuning()

In [ ]:
ParamTuning()

In [ ]:
ParamTuning()